Why Summarize Texts
There are a number of valid reasons in favor of the automatic summarization of documents:

Каких же?

In general, summarization algorithms are either **extractive** or **abstractive** based on the summary generated. 


Extractive algorithms form summaries by identifying and pasting together relevant sections of the input text. 

bstractive algorithms are most human-like and mimic the process of paraphrasing a text, which may generate new text that is not present in the initial document. Texts summarized using this technique looks more human-like and produces condensed summaries. 

Как вы думаете, в чём плюсы и минусы каждого?


Thus, they depend only on extraction of sentences from the original text. For such a reason, extractive methods yield naturally grammatical summaries and require relatively little linguistic analysis.

In [30]:
! pip install -U spacy
! pip install -U scikit-learn

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.3.2)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)


In [31]:
! python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


https://spacy.io/usage/models (зайдём?)

nlp = spacy.load('en_core_web_sm')

Будем пользоваться этой моделью. Для русского нету (официально).
гит

In [9]:
from google.colab import files

uploaded = files.upload()

Saving text.txt to text.txt


In [32]:
import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer

with open("text.txt", "r", encoding="utf-8") as f:
        text = " ".join(f.readlines())
doc = nlp(text)        


The text is further divided into sentences and then we convert them to a matrix of token counts. Next, we create a dictionary that contains the extracted words and their respective frequencies. Besides, we discard unnecessary words like a, an, the, is, of, and others, which are known as stop words. These are high-frequency words that do not carry any information and don’t serve any purpose towards our goal of summarization.

In [ ]:
#Уберем стоп слова и создадим словарь по схеме ниже ( с relative frequency от нуля до единицы)

In [37]:
word_frequency

{'000': 0.13333333333333333,
 '07': 0.06666666666666667,
 '10pm': 0.06666666666666667,
 '30': 0.06666666666666667,
 '31': 0.06666666666666667,
 '33': 0.06666666666666667,
 '37': 0.06666666666666667,
 '40s': 0.06666666666666667,
 '99': 0.06666666666666667,
 'able': 0.13333333333333333,
 'accident': 0.06666666666666667,
 'aches': 0.06666666666666667,
 'admiration': 0.06666666666666667,
 'admit': 0.06666666666666667,
 'advisory': 0.06666666666666667,
 'age': 0.06666666666666667,
 'ago': 0.06666666666666667,
 'amazing': 0.13333333333333333,
 'anxiety': 0.06666666666666667,
 'anxious': 0.06666666666666667,
 'appetite': 0.06666666666666667,
 'areas': 0.06666666666666667,
 'aren': 0.06666666666666667,
 'associate': 0.13333333333333333,
 'august': 0.13333333333333333,
 'avalanche': 0.06666666666666667,
 'awake': 0.06666666666666667,
 'away': 0.06666666666666667,
 'backache': 0.06666666666666667,
 'barnes': 0.06666666666666667,
 'base': 0.06666666666666667,
 'bathroom': 0.06666666666666667,
 'b

Almost all extractive summarization methods have three main obstacles.

 The first one is how you rank sentences or words. ____Идеи?

 The second obstacle is how to select the ranked units. ___Какие тут могут быть проблемы и как их решать?

 The third obstacle is how to ensure that the selected units form an understandable summary rather than being a set of disconnected words or sentences.  ____А вот это сложно и отдельная NLP тема.
 
 
 To solve the ranking problem, we will use a simple algorithm that determines the relevance of sentences based on the cumulative frequency of their words. Then, firstly we compute the relative frequency of each word:

Каким путём они тут пошли?

In [27]:
val=sorted(word_frequency.values())
higher_word_frequencies = [word for word,freq in word_frequency.items() if freq in val[-3:]]
print("\nWords with higher frequencies: ", higher_word_frequencies)
# gets relative frequency of words
higher_frequency = val[-1]
for word in word_frequency.keys():  
    word_frequency[word] = (word_frequency[word]/higher_frequency)


Words with higher frequencies:  ['staff', 'time', 've']


In [43]:
sentence_rank={}
for sent in doc.sents:
    for word in sent :       
        if word.text.lower() in word_frequency.keys():            
            if sent in sentence_rank.keys():
                sentence_rank[sent]+=word_frequency[word.text.lower()]
            else:
                sentence_rank[sent]=word_frequency[word.text.lower()]
top_sentences=(sorted(sentence_rank.values())[::-1])
top_sent=top_sentences[:6]

А можете написать код с ключевыми словами? Потом сравним)

In [ ]:
#YouRCODE

In [44]:
summary=[]
for sent,strength in sentence_rank.items():  
    if strength in top_sent:
        summary.append(sent)
    else:
        continue
for i in summary:
    print(i,end=" ")

At the beginning, we didn't know how brutal the impact of Covid would be, and the toll it would take watching people you've got to know over a matter of days or weeks just deteriorate. I went to live in a flat in Bradford for the first few months of the pandemic to be nearer the hospital, and because it was safer - my partner has multiple sclerosis and my daughter has learning difficulties and special needs, and I didn't want to take Covid back home. I started living there weekdays after I ended up driving in at 10pm at night because we were opening a red ward (for infectious Covid patients) at very short notice. We have staff who have moved out to live elsewhere in order to protect relatives at home, staff who have cancelled holidays, who aren't seeing their loved ones because they're dedicated to working here The counselling we were getting ended in August, but in the last two weeks it started again, because I had staff just crying - staff texting me to say they're crying in the car,

В чём "минусы" нашего текста для анализа? Как это можно поправить?
Какой "идеальный" текст для такой метрики?


# ОТВЕТЫ

In [26]:
corpus = [sent.text.lower() for sent in doc.sents ]
cv = CountVectorizer(stop_words=list(STOP_WORDS))   
cv_fit=cv.fit_transform(corpus)    
word_list = cv.get_feature_names();    
count_list = cv_fit.toarray().sum(axis=0)
word_frequency = dict(zip(word_list,count_list))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


### Теор, нейро и Latent Semantic Analysis

https://www.researchgate.net/publication/220195824_Text_summarization_using_Latent_Semantic_Analysis    -- обзорная статья (история, классификация....)

Другие подходы:

Концепты

Как эти концепты определить?


  LSA (Latent Semantic Analysis) assumes that words that are close in meaning will occur in similar pieces of text (the distributional hypothesis). A matrix containing word counts per document (rows represent unique words and columns represent each document) is constructed from a large piece of text and a mathematical technique called singular value decomposition (SVD) is used to reduce the number of rows while preserving the similarity structure among columns. 


  Documents are then compared by taking the cosine of the angle between the two vectors (or the dot product between the normalizations of the two vectors) formed by any two columns. Values close to 1 represent very similar documents while values close to 0 represent very dissimilar documents.

Какие недостатки? 

The approach of Gong and Liu has some disadvantages that are defined by Steinberger and Jezek [5]. The first disad-vantage is that the number of sentences to be collected is the same with the reduced dimension. If the given predefinednumber is large, sentences from less significant concepts are chosen. The second disadvantage is related to choosingonly one sentence from each concept. Some concepts, especially important ones, can contain sentences that are highlyrelated to the concept, but do not have the highest cell value. The last disadvantage is that all chosen concepts areassumed to be in the same importance level, which may not be true.н

 Steinberger and Jezek (2004) и Ozsoy et al - способ вычленить не по одному, а по нескольку предложений в зависимости от релевантности топика (SVD разложение).[Текст ссылки](https://)

+ некоторые работы предлагают смотреть на длину предложения (почему эту переменную надо использовать аккуратно?)

##  Теперь чуть-чуть про более продвинутые виды.

Encoder-Decoder, Seq to Seq model.

Очень прогрессивно, но пока что не очень адекватно - сами по себе они не работают.

https://github.com/lancopku/Global-Encoding

 Latent Semantic AnalysisLatent Semantic Analysis is an algebraic-statistical method that extracts hidden semantic structures of words and sen-tences. It is an unsupervised approach